# Train a Convolutional Cell Complex Network (CCXN)

We create and train a simplified version of the CCXN originally proposed in [Hajij et. al : Cell Complex Neural Networks (2020)](https://arxiv.org/pdf/2010.00743.pdf).

### The Neural Network:

The equations of one layer of this neural network are given by:

1. A convolution from nodes to nodes using an adjacency message passing scheme (AMPS):

🟥 $\quad m_{y \rightarrow \{z\} \rightarrow x}^{(0 \rightarrow 1 \rightarrow 0)} = M_{\mathcal{L}_\uparrow}^t(h_x^{t,(0)}, h_y^{t,(0)}, \Theta^{t,(y \rightarrow x)})$ 

🟧 $\quad m_x^{(0 \rightarrow 1 \rightarrow 0)} = AGG_{y \in \mathcal{L}_\uparrow(x)}(m_{y \rightarrow \{z\} \rightarrow x}^{0 \rightarrow 1 \rightarrow 0})$ 

🟩 $\quad m_x^{(0)} = m_x^{(0 \rightarrow 1 \rightarrow 0)}$ 

🟦 $\quad h_x^{t+1,(0)} = U^{t}(h_x^{t,(0)}, m_x^{(0)})$

2. A convolution from edges to faces using a cohomology message passing scheme:

🟥 $\quad m_{y \rightarrow x}^{(r' \rightarrow r)} = M^t_{\mathcal{C}}(h_{x}^{t,(r)}, h_y^{t,(r')}, x, y)$ 

🟧 $\quad m_x^{(r' \rightarrow r)}  = AGG_{y \in \mathcal{C}(x)} m_{y \rightarrow x}^{(r' \rightarrow r)}$ 

🟩 $\quad m_x^{(r)} = m_x^{(r' \rightarrow r)}$ 

🟦 $\quad h_{x}^{t+1,(r)} = U^{t,(r)}(h_{x}^{t,(r)}, m_{x}^{(r)})$

Where the notations are defined in [Papillon et al : Architectures of Topological Deep Learning: A Survey of Topological Neural Networks (2023)](https://arxiv.org/abs/2304.10031).

### The Task:

We train this model to perform entire complex classification on [`MUTAG` from the TUDataset](https://paperswithcode.com/dataset/mutag). This dataset contains:
- 188 samples of chemical compounds represented as graphs,
- with 7 discrete node features.

The task is to predict the mutagenicity of each compound on Salmonella typhimurium.

# Set-up


In [1]:
import random

import numpy as np
import torch
from sklearn.model_selection import train_test_split
from toponetx import CellComplex
from torch_geometric.datasets import TUDataset
from torch_geometric.utils.convert import to_networkx

from topomodelx.nn.cell.can_layer import CANLayer

If GPU's are available, we will make use of them. Otherwise, this will run on CPU.

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


# Pre-processing

## Import data ##

We import a subset of MUTAG, a benchmark dataset for graph classification. 

We then lift each graph into our topological domain of choice, here: a cell complex.

We also retrieve:
- input signals `x_0` and `x_1` on the nodes (0-cells) and edges (1-cells) for each complex: these will be the model's inputs,
- a binary classification label `y` associated to the cell complex.

In [3]:
dataset = TUDataset(
    root="/tmp/MUTAG", name="MUTAG", use_edge_attr=True, use_node_attr=True
)
dataset = dataset[:20]
cc_list = []
x_0_list = []
x_1_list = []
y_list = []
for graph in dataset:
    cell_complex = CellComplex(to_networkx(graph))
    cc_list.append(cell_complex)
    x_0_list.append(graph.x)
    x_1_list.append(graph.edge_attr)
    y_list.append(int(graph.y))

i_cc = 0
print(f"Features on nodes for the {i_cc}th cell complex: {x_0_list[i_cc].shape}.")
print(f"Features on edges for the {i_cc}th cell complex: {x_1_list[i_cc].shape}.")
print(f"Label of {i_cc}th cell complex: {y_list[i_cc]}.")

Features on nodes for the 0th cell complex: torch.Size([17, 7]).
Features on edges for the 0th cell complex: torch.Size([38, 4]).
Label of 0th cell complex: 1.


In [4]:
for graph in dataset:
    cell_complex = CellComplex(to_networkx(graph))
    x = cell_complex.down_laplacian_matrix(rank=1)
    try:
        x = cell_complex.up_laplacian_matrix(rank=1)
    except:
        x = torch.zeros((x.shape[0], x.shape[0]))
    print(x.shape)
    break

torch.Size([38, 38])


## Define neighborhood structures. ##

Implementing the CCXN architecture will require to perform message passing along neighborhood structures of the cell complexes.

Thus, now we retrieve these neighborhood structures (i.e. their representative matrices) that we will use to send messages. 

For the CCXN, we need the adjacency matrix $A_{\uparrow, 0}$ and the coboundary matrix $B_2^T$ of each cell complex.

In [5]:
lower_neighborhood_list = []
upper_neighborhood_list = []

for cell_complex in cc_list:
    lower_neighborhood_t = cell_complex.down_laplacian_matrix(rank=1)
    lower_neighborhood_t = torch.from_numpy(lower_neighborhood_t.todense()).to_sparse()
    lower_neighborhood_list.append(lower_neighborhood_t)

    try:
        upper_neighborhood_t = cell_complex.up_laplacian_matrix(rank=1)
        upper_neighborhood_t = torch.from_numpy(upper_neighborhood_t.todense()).to_sparse()
    except:
        upper_neighborhood_t = np.zeros((lower_neighborhood_t.shape[0], lower_neighborhood_t.shape[0]))
        upper_neighborhood_t = torch.from_numpy(upper_neighborhood_t).to_sparse()

    
    upper_neighborhood_list.append(upper_neighborhood_t)

i_cc = 0

# Create the Neural Network

Using the CCXNLayer class, we create a neural network with stacked layers.

In [6]:
in_channels_0 = x_0_list[0].shape[-1]
in_channels_1 = x_1_list[0].shape[-1]
in_channels_2 = 5
print(
    f"The dimension of input features on nodes, edges and faces are: {in_channels_0}, {in_channels_1} and {in_channels_2}."
)

The dimension of input features on nodes, edges and faces are: 7, 4 and 5.


In [7]:
class CAN(torch.nn.Module):
    """CAN.

    Parameters
    ----------
    """

    def __init__(
        self,
        in_channels,
        out_channels,
        num_classes,
        n_layers=2,
        att=False,
    ):
        super().__init__()
        
        layers = []

        layers.append(                
            CANLayer(
                    in_channels=in_channels,
                    out_channels=out_channels,
                    aggr_func="sum",
                    update_func="relu"
                )
        )

        for _ in range(n_layers-1):
            layers.append(
                CANLayer(
                    in_channels=out_channels,
                    out_channels=out_channels,
                    aggr_func="sum",
                    update_func="relu"
                )
            )
        self.layers = torch.nn.ModuleList(layers)
        self.lin_0 = torch.nn.Linear(out_channels, 1024)
        self.lin_1 = torch.nn.Linear(1024, num_classes)

    def forward(self, x, lower_neighborhood, upper_neighborhood):
        """Forward computation through layers, then linear layers, then avg pooling.

        Parameters
        ----------


        Returns
        -------

        """
        for layer in self.layers:
            x = layer(x, lower_neighborhood, upper_neighborhood)
            

        # max pooling over all nodes in each graph
        x = x.max(dim=0)[0]

        # FF-NN
        out = self.lin_1(torch.nn.functional.relu(self.lin_0(x)))

        return out

# Train the Neural Network

We specify the model, initialize loss, and specify an optimizer. We first try it without any attention mechanism.

In [8]:
model = CAN(in_channels_1, 32, num_classes=2, n_layers=1)
model = model.to(device)
crit = torch.nn.CrossEntropyLoss()
opt = torch.optim.Adam(model.parameters(), lr=0.001)

We split the dataset into train and test sets.

In [9]:
test_size = 0.3
x_1_train, x_1_test = train_test_split(x_1_list, test_size=test_size, shuffle=False)
lower_neighborhood_train, lower_neighborhood_test = train_test_split(
    lower_neighborhood_list, test_size=test_size, shuffle=False
)
upper_neighborhood_train, upper_neighborhood_test = train_test_split(
    upper_neighborhood_list, test_size=test_size, shuffle=False
)
y_train, y_test = train_test_split(y_list, test_size=test_size, shuffle=False)

We train the CCXN using low amount of epochs: we keep training minimal for the purpose of rapid testing.

In [10]:
test_interval = 2
num_epochs = 50
for epoch_i in range(1, num_epochs + 1):
    epoch_loss = []
    num_samples = 0
    correct = 0
    model.train()
    for x_1, lower_neighborhood, upper_neighborhood, y in zip(
        x_1_train, lower_neighborhood_train, upper_neighborhood_train, y_train
    ):
        x_1, y = x_1.float().to(device), torch.tensor(y, dtype=torch.long).to(device)
        lower_neighborhood, upper_neighborhood = lower_neighborhood.float().to(device), upper_neighborhood.float().to(device)
        opt.zero_grad()
        y_hat = model(x_1, lower_neighborhood, upper_neighborhood)

        loss = crit(y_hat, y)
        correct += (y_hat.argmax() == y).sum().item()
        num_samples += 1
        loss.backward()
        opt.step()
        epoch_loss.append(loss.item())
    train_acc = correct / num_samples
    print(
        f"Epoch: {epoch_i} loss: {np.mean(epoch_loss):.4f} Train_acc: {train_acc:.4f}",
        flush=True,
    )
    if epoch_i % test_interval == 0:
        with torch.no_grad():
            num_samples = 0
            correct = 0
            for x_1, lower_neighborhood, upper_neighborhood, y in zip(
                x_1_test, lower_neighborhood_test, upper_neighborhood_test, y_test
            ):
                x_1, y = x_1.float().to(device), torch.tensor(y, dtype=torch.long).to(device)
                lower_neighborhood, upper_neighborhood = lower_neighborhood.float().to(device), upper_neighborhood.float().to(device)
                y_hat = model(x_1, lower_neighborhood, upper_neighborhood)
                correct += (y_hat.argmax() == y).sum().item()
                num_samples += 1
            test_acc = correct / num_samples
            print(f"Test_acc: {test_acc:.4f}", flush=True)

Epoch: 1 loss: 0.7362 Train_acc: 0.2857
Epoch: 2 loss: 0.6698 Train_acc: 0.5714
Test_acc: 0.6667
Epoch: 3 loss: 0.6654 Train_acc: 0.7143
Epoch: 4 loss: 0.6479 Train_acc: 0.7143
Test_acc: 0.6667
Epoch: 5 loss: 0.6334 Train_acc: 0.7143
Epoch: 6 loss: 0.6237 Train_acc: 0.7143
Test_acc: 0.6667
Epoch: 7 loss: 0.6160 Train_acc: 0.7143
Epoch: 8 loss: 0.6085 Train_acc: 0.7143
Test_acc: 0.6667
Epoch: 9 loss: 0.6025 Train_acc: 0.7143
Epoch: 10 loss: 0.5973 Train_acc: 0.7143
Test_acc: 0.6667
Epoch: 11 loss: 0.5931 Train_acc: 0.7143
Epoch: 12 loss: 0.5889 Train_acc: 0.7143
Test_acc: 0.6667
Epoch: 13 loss: 0.5865 Train_acc: 0.7143
Epoch: 14 loss: 0.5824 Train_acc: 0.7143
Test_acc: 0.6667
Epoch: 15 loss: 0.5802 Train_acc: 0.7143
Epoch: 16 loss: 0.5777 Train_acc: 0.7143
Test_acc: 0.6667
Epoch: 17 loss: 0.5742 Train_acc: 0.7143
Epoch: 18 loss: 0.5725 Train_acc: 0.7143
Test_acc: 0.6667
Epoch: 19 loss: 0.5703 Train_acc: 0.7143
Epoch: 20 loss: 0.5685 Train_acc: 0.7143
Test_acc: 0.6667
Epoch: 21 loss: 0.5